In [ ]:
# Libraries 
import cv2
import mediapipe as mp
from math import hypot
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

# Initializing Webcam and Hand Tracking
cap = cv2.VideoCapture(0) # Access the default webcam (0)
mpHands = mp.solutions.hands
hands = mpHands.Hands() # Initialize hand tracking model
mpDraw = mp.solutions.drawing_utils # For drawing landmarks on hands

# Accessing System Volume Control
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volMin, volMax = volume.GetVolumeRange()[:2]

# Main Body
while True:
    success, img = cap.read()  # Capture frame-by-frame
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    results = hands.process(imgRGB)  # Process image for hand detection
    lmList = []
    if results.multi_hand_landmarks:  # If hands are detected
        # Loop through each hand and extract landmark coordinates
        for handlandmark in results.multi_hand_landmarks:
            for id, lm in enumerate(handlandmark.landmark):
                h, w, _ = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])
            mpDraw.draw_landmarks(img, handlandmark, mpHands.HAND_CONNECTIONS)
        # If landmarks are detected
        if lmList != []:
            x1, y1 = lmList[4][1], lmList[4][2]  # Thumb tip coordinates
            x2, y2 = lmList[8][1], lmList[8][2]  # Index finger tip coordinates
            # Draw circles and line connecting landmarks
            cv2.circle(img, (x1, y1), 4, (255, 0, 0), cv2.FILLED)
            cv2.circle(img, (x2, y2), 4, (255, 0, 0), cv2.FILLED)
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
            length = hypot(x2 - x1, y2 - y1)  # Calculate distance between landmarks
            # Map distance to volume range and set volume level
            vol = np.interp(length, [15, 220], [volMin, volMax])
            volume.SetMasterVolumeLevel(vol, None)
    cv2.imshow('Image', img)  # Display the image
    if cv2.waitKey(1) & 0xff == ord('q'):  # Press 'q' to quit
        break